In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    criterion: str
    max_depth: int
    n_estimators: int
    oob_score: bool
    target_column: int

In [6]:

from Phishingproject.constants import *
from Phishingproject.utils.common import read_yaml_file, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        self.schema = read_yaml_file(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            criterion = params.criterion,
            max_depth = params.max_depth,
            n_estimators = params.n_estimators,
            oob_score = params.oob_score,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from Phishingproject.logging import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = RandomForestClassifier(criterion=self.config.criterion,max_depth=self.config.max_depth,n_estimators=self.config.n_estimators,oob_score=self.config.oob_score, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_model_trainer_config()
    data_transformation = ModelTrainer(config=data_transformation_config)
    data_transformation.train()
except Exception as e:
    raise e

[2024-07-10 00:38:54,098: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-07-10 00:38:54,103: INFO:common: yaml file: params.yaml loaded successfully]
[2024-07-10 00:38:54,110: INFO:common: yaml file: schema.yaml loaded successfully]
[2024-07-10 00:38:54,117: INFO:common: createD Directory at:artifacts]
[2024-07-10 00:38:54,119: INFO:common: createD Directory at:artifacts/model_trainer]


c:\Users\suzal\anaconda3\envs\pcenv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
